** Developing an AI application **

Going forward, AI algorithms will be incorporated into more and more everyday applications. For example, you might want to include an image classifier in a smart phone app. To do this, you'd use a deep learning model trained on hundreds of thousands of images as part of the overall application architecture. A large part of software development in the future will be using these types of models as common parts of applications.

In this project, you'll train an image classifier to recognize different species of flowers. You can imagine using something like this in a phone app that tells you the name of the flower your camera is looking at. In practice you'd train this classifier, then export it for use in your application. We'll be using this dataset of 102 flower categories, you can see a few examples below.

The project is broken down into multiple steps:

* Load and preprocess the image dataset
* Train the image classifier on your dataset
* Use the trained classifier to predict image content
* We'll lead you through each part which you'll implement in Python.

When you've completed this project, you'll have an application that can be trained on any set of labeled images. Here your network will be learning about flowers and end up as a command line application. But, what you do with your new skills depends on your imagination and effort in building a dataset. For example, imagine an app where you take a picture of a car, it tells you what the make and model is, then looks up information about it. Go build your own dataset and make something new.

First up is importing the packages you'll need. It's good practice to keep all the imports at the beginning of your code. As you work through this notebook and find you need to import a package, make sure to add the import up here.

In [0]:
# google colab does not come with torch installed. And also, in course we are using torch 0.4. 
# so following snippet of code installs the relevant version

from google.colab import drive
drive.mount('/content/gdrive')

from os.path import exists
from wheel.pep425tags import get_abbr_impl, get_impl_ver, get_abi_tag
platform = '{}{}-{}'.format(get_abbr_impl(), get_impl_ver(), get_abi_tag())
cuda_output = !ldconfig -p|grep cudart.so|sed -e 's/.*\.\([0-9]*\)\.\([0-9]*\)$/cu\1\2/'
accelerator = cuda_output[0] if exists('/dev/nvidia0') else 'cpu'

!pip install -q http://download.pytorch.org/whl/{accelerator}/torch-0.4.1-{platform}-linux_x86_64.whl torchvision
import torch


In [0]:
# we will verify that GPU is enabled for this notebook
# following should print: CUDA is available!  Training on GPU ...
# 
# if it prints otherwise, then you need to enable GPU: 
# from Menu > Runtime > Change Runtime Type > Hardware Accelerator > GPU

import torch
import numpy as np

# check if CUDA is available
train_on_gpu = torch.cuda.is_available()

if not train_on_gpu:
    print('CUDA is not available.  Training on CPU ...')
else:
    print('CUDA is available!  Training on GPU ...')

CUDA is available!  Training on GPU ...


In [0]:
# we need pillow version of 5.3.0
# we will uninstall the older version first
!pip uninstall -y Pillow
# install the new one
!pip install Pillow==5.3.0
# import the new one
import PIL
print(PIL.PILLOW_VERSION)
# this should print 5.3.0. If it doesn't, then restart your runtime:
# Menu > Runtime > Restart Runtime

Uninstalling Pillow-5.3.0:
  Successfully uninstalled Pillow-5.3.0
  Using cached https://files.pythonhosted.org/packages/62/94/5430ebaa83f91cc7a9f687ff5238e26164a779cca2ef9903232268b0a318/Pillow-5.3.0-cp36-cp36m-manylinux1_x86_64.whl
5.3.0


In [0]:
# Imports here
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import time
import json
import copy

import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from PIL import Image
from collections import OrderedDict


import torch
from torch import nn, optim
import torch.nn.functional as F
from torch.optim import lr_scheduler
from torch.autograd import Variable
from torchvision import datasets, models, transforms

** Load the data **

Here you'll use torchvision to load the data (documentation). You can download the data here. The dataset is split into two parts, training and validation. For the training, you'll want to apply transformations such as random scaling, cropping, and flipping. This will help the network generalize leading to better performance. If you use a pre-trained network, you'll also need to make sure the input data is resized to 224x224 pixels as required by the networks.

The validation set is used to measure the model's performance on data it hasn't seen yet. For this you don't want any scaling or rotation transformations, but you'll need to resize then crop the images to the appropriate size.

In [0]:
# we will download the required data files
!wget -cq https://github.com/udacity/pytorch_challenge/raw/master/cat_to_name.json
!wget -cq https://s3.amazonaws.com/content.udacity-data.com/courses/nd188/flower_data.zip
!rm -r flower_data || true
!unzip -qq flower_data.zip

In [0]:
data_dir = './flower_data'
train_dir = data_dir + '/train'
valid_dir = data_dir + '/valid'

In [0]:
# TODO: Define your transforms for the training and validation sets
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomRotation(30),
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], 
                             [0.229, 0.224, 0.225])
    ]),
    'valid': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], 
                             [0.229, 0.224, 0.225])
    ])
}


# TODO: Load the datasets with ImageFolder

#train_data = datasets.ImageFolder(train_dir, transform = train_transforms)
#validation_data = datasets.ImageFolder(valid_dir, transform = validation_transforms)
dirs = {'train': train_dir, 
        'valid': valid_dir}

image_datasets = {x: datasets.ImageFolder(dirs[x], transform=data_transforms[x]) for x in ['train', 'valid']}


# TODO: Using the image datasets and the trainforms, define the dataloaders
#train_loader = torch.utils.data.DataLoader(train_data, batch_size = 32, shuffle = True)
#validation_loader = torch.utils.data.DataLoader(validation_data, batch_size = 32, shuffle = True)
dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=32, shuffle=True) 
               for x in ['train', 'valid']}


dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'valid']}
class_names = image_datasets['train'].classes
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

** Label mapping  **

You'll also need to load in a mapping from category label to category name. You can find this in the file cat_to_name.json. It's a JSON object which you can read in with the json module. This will give you a dictionary mapping the integer encoded categories to the actual names of the flowers.

In [0]:
import json

with open('cat_to_name.json', 'r') as f:
    cat_to_name = json.load(f)

In [0]:
# TODO: Build and train your network
model = models.vgg19(pretrained = True)

In [0]:
# Custom classifier to replace vgg19 classifier
classifier = nn.Sequential(OrderedDict([
                          ('fc1', nn.Linear(25088, 4096)),
                          ('relu', nn.ReLU()),
                          ('fc2', nn.Linear(4096, 102)),
                          ('output', nn.LogSoftmax(dim=1))
                          ]))

In [0]:
# Stop updating of pre-trained models
for param in model.parameters():
    param.requires_grad = False

    
# Replace vgg19 classifier with custom classifier
model.classifier = classifier

In [0]:
# Training Model

def training_model(model, criteria, optimizer, scheduler, epochs = 30):
    
    
    
    #wts_best_model = copy.deepcopy(model.state.dict())
    best_acc = 0.0
    
    model.to(device)
    
    for e in range(epochs):
        print('Epoch {}/{}'.format(e + 1, epochs))
        print('-' * 12)
        
        for phase in ['train', 'valid']:
            if phase == 'train':
                scheduler.step()
                model.train()
            else:
                model.eval()
                
            running_loss = 0.0
            correct_predictions = 0
            
            # Iterate over data
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)
                
                # Clear parameter gradients
                optimizer.zero_grad()
                
                #forward propagation
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)
                    
                    #backward propagation and optimization
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()
                        
                # Calculate Statistics
                running_loss += loss.item() * inputs.size(0)
                correct_predictions += torch.sum(preds == labels.data)
                
            
            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = correct_predictions.double() / dataset_sizes[phase]
            
            
            print('{} Loss: {:.4f} Acc: {:.4f}'.format(
                phase, epoch_loss, epoch_acc))

            # save deep model
            if phase == 'valid' and epoch_acc > best_acc:
                best_acc = epoch_acc
                wts_best_model = copy.deepcopy(model.state_dict())
        
        print()
        
    print('Best val Acc: %d%%' %(best_acc * 100))

    # load best model weights
    model.load_state_dict(wts_best_model)
        
    return model

        

In [0]:
# Hyperparameters

# Criteria for Softmax final layer 
criterion = nn.NLLLoss()

#Optimizer
#optimizer = optim.Adam(model.classifier.parameters(), lr=0.003)
optimizer = optim.SGD(model.classifier.parameters(), lr = 0.01, momentum=0.9)

# Learning rate decay
sched = lr_scheduler.StepLR(optimizer, step_size = 5, gamma = 0.1)

# Number of Epochs

eps = 15

In [0]:
train_model = training_model(model, criterion, optimizer, sched, eps)

Epoch 1/15
------------
train Loss: 2.3998 Acc: 0.4245
valid Loss: 1.2692 Acc: 0.6760

Epoch 2/15
------------
train Loss: 1.3570 Acc: 0.6358
valid Loss: 0.8938 Acc: 0.7396

Epoch 3/15
------------
train Loss: 1.1165 Acc: 0.6944
valid Loss: 0.8468 Acc: 0.7800

Epoch 4/15
------------
train Loss: 1.0121 Acc: 0.7300
valid Loss: 0.7781 Acc: 0.8227

Epoch 5/15
------------
train Loss: 0.9505 Acc: 0.7494
valid Loss: 0.6920 Acc: 0.8240

Epoch 6/15
------------
train Loss: 0.6037 Acc: 0.8367
valid Loss: 0.4319 Acc: 0.8814

Epoch 7/15
------------
train Loss: 0.5112 Acc: 0.8576
valid Loss: 0.4013 Acc: 0.8961

Epoch 8/15
------------
train Loss: 0.4818 Acc: 0.8684
valid Loss: 0.3820 Acc: 0.8973

Epoch 9/15
------------
train Loss: 0.4314 Acc: 0.8777
valid Loss: 0.3606 Acc: 0.9010

Epoch 10/15
------------
train Loss: 0.4323 Acc: 0.8794
valid Loss: 0.3656 Acc: 0.9010

Epoch 11/15
------------
train Loss: 0.4090 Acc: 0.8829
valid Loss: 0.3524 Acc: 0.9034

Epoch 12/15
------------
train Loss: 0.38

In [0]:
# TODO: Save the checkpoint 

model.class_to_idx = image_datasets['train'].class_to_idx
model.cpu()

torch.save({
            'structure': 'vgg19',
            'state_dict': model.state_dict(),
            'optimizer.state_dict': optimizer.state_dict(),
            'epoch': eps,
            'class_to_idx': model.class_to_idx}, 'checkpoint.pth')

In [0]:
# TODO: Write a function that loads a checkpoint and rebuilds the model

def load_model(path):
    checkpoint = torch.load(path)
    
    model = models.vgg19(pretrained = True)
    for param in model.parameters():
        param.requires_grad = False
    
    model.class_to_idx = checkpoint['class_to_idx']
    
    # Classifier
    classifier = nn.Sequential(OrderedDict([
                          ('fc1', nn.Linear(25088, 4096)),
                          ('relu', nn.ReLU()),
                          ('fc2', nn.Linear(4096, 102)),
                          ('output', nn.LogSoftmax(dim=1))
                          ]))
    
    model.classifier = classifier
    model.load_state_dict(checkpoint['state_dict'])
    
    return model

In [0]:
def process_image(image):
    ''' Scales, crops, and normalizes a PIL image for a PyTorch model,
        returns an Numpy array
    '''
    
    # TODO: Process a PIL image for use in a PyTorch model
    
    #Open image
    img = Image.open(image)
    
    #Image Transformation
    img_transformation = transforms.Compose([
                                        transforms.Resize(256),
                                        transforms.CenterCrop(224),
                                        transforms.ToTensor(),
                                        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
                                    ])
    
    img = img_transformation(img)
    
    return img
    
    

In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive
